# DST

## tableinfo

In [94]:
import requests
json_body = {
   "table": "BYGV88"
}
result = requests.post("https://api.statbank.dk/v1/tableinfo",json = json_body).json()
result

{'id': 'BYGV88',
 'text': 'Det samlede etageareal (korrigeret for forsinkelser)',
 'description': 'Det samlede etageareal (korrigeret for forsinkelser) efter byggefase, anvendelse, bygherreforhold, sæsonkorrigering og tid',
 'unit': 'm2',
 'suppressedDataValue': '0',
 'updated': '2021-05-12T08:00:00',
 'active': True,
 'contacts': [{'name': 'Paul Lubson',
   'phone': '+4539173542',
   'mail': 'pal@dst.dk'}],
 'documentation': {'id': 'db96f09a-e616-40cb-ab0c-00eebff9ca6f',
  'url': 'https://www.dst.dk/statistikdokumentation/db96f09a-e616-40cb-ab0c-00eebff9ca6f'},
 'footnote': {'text': 'Overgangen til den nye version af BBR (ny struktur og nye metoder) i juni 2017 betød i forbindelse med offentliggørelsen af 4. kvartal 2017 og 1. kvartal 2018 store revisioner af alle byggestatistikkens tidsserier. For opgørelserne af tilladt og især påbegyndt byggeri er der dog stadig en betydelig usikkerhed om tallene for det seneste, løbende 1½ år. Det forventes, at der senere vil komme væsentlige opju

## Data

In [95]:
variables_ = []
for item in result['variables']:
    code = item['id']
    values = [value['id'] for value in item['values']]
    variables_.append( {'code':code,'values':values} )

In [96]:
import pandas as pd
from io import StringIO

json_body = {
   "table": "BYGV88",
   "format": "CSV",
   "timeOrder": "Ascending",
   "variables": variables_
}
result = requests.post("https://api.statbank.dk/v1/data",json = json_body)
pd.read_csv(StringIO(result.text),sep=";")

,BYGFASE,ANVENDELSE,BYGHERRE,SÆSON,TID,INDHOLD
0,Påbegyndt byggeri,Anvendelse i alt,I alt,Ikke sæsonkorrigeret,1998M01,570555
1,Påbegyndt byggeri,Anvendelse i alt,I alt,Ikke sæsonkorrigeret,1998M02,607638
2,Påbegyndt byggeri,Anvendelse i alt,I alt,Ikke sæsonkorrigeret,1998M03,858924
3,Påbegyndt byggeri,Anvendelse i alt,I alt,Ikke sæsonkorrigeret,1998M04,860835
4,Påbegyndt byggeri,Anvendelse i alt,I alt,Ikke sæsonkorrigeret,1998M05,863822
...,...,...,...,...,...,...
35707,Tilladt byggeri,Øvrige bygninger,Offentlige bygninger,Sæsonkorrigeret,2020M11,0
35708,Tilladt byggeri,Øvrige bygninger,Offentlige bygninger,Sæsonkorrigeret,2020M12,0
35709,Tilladt byggeri,Øvrige bygninger,Offentlige bygninger,Sæsonkorrigeret,2021M01,0
35710,Tilladt byggeri,Øvrige bygninger,Offentlige bygninger,Sæsonkorrigeret,2021M02,0


# As a function

In [101]:
def get_dst_dataframe(table_id,parse_date=True):
    import requests
    import pandas as pd
    from io import StringIO
    from datetime import datetime

    json_body = {
    "table": table_id
    }
    result_meta = requests.post("https://api.statbank.dk/v1/tableinfo",json = json_body).json()
    result = requests.post("https://api.statbank.dk/v1/tableinfo",json = json_body).json()
    # Build variable
    variables_ = []
    for item in result['variables']:
        code = item['id']
        values = [value['id'] for value in item['values']]
        variables_.append( {'code':code,'values':values} )
    json_body = {
        "table": table_id,
        "format": "CSV",
        "timeOrder": "Ascending",
        "variables": variables_
    }
    result = requests.post("https://api.statbank.dk/v1/data",json = json_body)
    result_df = pd.read_csv(StringIO(result.text),sep=";")
    # Parse date
    if parse_date:
        # Month
        if 'M' in result_df.iloc[0]['TID']:
            result_df['TID'] = result_df['TID'].apply(lambda x: datetime.strptime(x, '%YM%m') )

    return result_df

In [98]:
get_dst_dataframe("BYGV88")

,BYGFASE,ANVENDELSE,BYGHERRE,SÆSON,TID,INDHOLD
0,Påbegyndt byggeri,Anvendelse i alt,I alt,Ikke sæsonkorrigeret,01-01-1998 00:00:00,570555
1,Påbegyndt byggeri,Anvendelse i alt,I alt,Ikke sæsonkorrigeret,01-02-1998 00:00:00,607638
2,Påbegyndt byggeri,Anvendelse i alt,I alt,Ikke sæsonkorrigeret,01-03-1998 00:00:00,858924
3,Påbegyndt byggeri,Anvendelse i alt,I alt,Ikke sæsonkorrigeret,01-04-1998 00:00:00,860835
4,Påbegyndt byggeri,Anvendelse i alt,I alt,Ikke sæsonkorrigeret,01-05-1998 00:00:00,863822
...,...,...,...,...,...,...
35707,Tilladt byggeri,Øvrige bygninger,Offentlige bygninger,Sæsonkorrigeret,01-11-2020 00:00:00,0
35708,Tilladt byggeri,Øvrige bygninger,Offentlige bygninger,Sæsonkorrigeret,01-12-2020 00:00:00,0
35709,Tilladt byggeri,Øvrige bygninger,Offentlige bygninger,Sæsonkorrigeret,01-01-2021 00:00:00,0
35710,Tilladt byggeri,Øvrige bygninger,Offentlige bygninger,Sæsonkorrigeret,01-02-2021 00:00:00,0


In [99]:
get_dst_dataframe("EJ13")

,REGION,EJENDOMSKATE,BNØGLE,TID,INDHOLD
0,Hele landet,Enfamiliehuse,Salg ved prisberegning (antal),01-01-2006 00:00:00,3676
1,Hele landet,Enfamiliehuse,Salg ved prisberegning (antal),01-02-2006 00:00:00,3923
2,Hele landet,Enfamiliehuse,Salg ved prisberegning (antal),01-03-2006 00:00:00,5004
3,Hele landet,Enfamiliehuse,Salg ved prisberegning (antal),01-04-2006 00:00:00,4035
4,Hele landet,Enfamiliehuse,Salg ved prisberegning (antal),01-05-2006 00:00:00,4890
...,...,...,...,...,...
8731,Region Nordjylland,"Ejerlejligheder, i alt",Beregnet antal salg,01-10-2020 00:00:00,..
8732,Region Nordjylland,"Ejerlejligheder, i alt",Beregnet antal salg,01-11-2020 00:00:00,..
8733,Region Nordjylland,"Ejerlejligheder, i alt",Beregnet antal salg,01-12-2020 00:00:00,..
8734,Region Nordjylland,"Ejerlejligheder, i alt",Beregnet antal salg,01-01-2021 00:00:00,..


In [100]:
get_dst_dataframe("EJ14")

,REGION,EJENDOMSKATE,TAL,TID,INDHOLD
0,Hele landet,Enfamiliehuse,Indeks,01-01-2006 00:00:00,"92,5"
1,Hele landet,Enfamiliehuse,Indeks,01-02-2006 00:00:00,"94,2"
2,Hele landet,Enfamiliehuse,Indeks,01-03-2006 00:00:00,"96,4"
3,Hele landet,Enfamiliehuse,Indeks,01-04-2006 00:00:00,"98,8"
4,Hele landet,Enfamiliehuse,Indeks,01-05-2006 00:00:00,"100,4"
...,...,...,...,...,...
6547,Region Nordjylland,"Ejerlejligheder, i alt",Ændring i forhold til samme måned året før (pct.),01-10-2020 00:00:00,..
6548,Region Nordjylland,"Ejerlejligheder, i alt",Ændring i forhold til samme måned året før (pct.),01-11-2020 00:00:00,..
6549,Region Nordjylland,"Ejerlejligheder, i alt",Ændring i forhold til samme måned året før (pct.),01-12-2020 00:00:00,..
6550,Region Nordjylland,"Ejerlejligheder, i alt",Ændring i forhold til samme måned året før (pct.),01-01-2021 00:00:00,..
